In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from difflib import get_close_matches

LEAGUE = "ESP-La Liga"
seasons = ['2018', '2019', '2020', '2021', '2022', '2023', '2024', '2025']

CACHE_DIR = Path("data_cache/football_data_spain")
CACHE_DIR.mkdir(parents=True, exist_ok=True)

TEAM_MAP_FD_TO_UNDERSTAT = {
    "Ath Madrid": "Atletico Madrid",
    "Ath Bilbao": "Athletic Club",
    "Betis": "Real Betis",
    "Sociedad": "Real Sociedad",
    "Valladolid": "Real Valladolid",
    "Espanol": "Espanyol",
    "Celta": "Celta Vigo",
    "Vallecano": "Rayo Vallecano",

    # comunes
    "Alaves": "Alaves",
    "Almeria": "Almeria",
    "Barcelona": "Barcelona",
    "Cadiz": "Cadiz",
    "Elche": "Elche",
    "Getafe": "Getafe",
    "Girona": "Girona",
    "Granada": "Granada",
    "Huesca": "Huesca",
    "Las Palmas": "Las Palmas",
    "Leganes": "Leganes",
    "Levante": "Levante",
    "Mallorca": "Mallorca",
    "Osasuna": "Osasuna",
    "Real Madrid": "Real Madrid",
    "Sevilla": "Sevilla",
    "Valencia": "Valencia",
    "Villarreal": "Villarreal",
}

def map_team_fd_to_understat(name: str) -> str:
    name = str(name).strip()
    return TEAM_MAP_FD_TO_UNDERSTAT.get(name, name)

def understat_season_to_fd_code(season_year: str) -> str:
    # '2018' -> '1819', ..., '2025' -> '2526'
    y = int(season_year)
    yy = y % 100
    yy2 = (y + 1) % 100
    return f"{yy:02d}{yy2:02d}"

def load_fd_sp1(season_code: str) -> pd.DataFrame:
    parquet_path = CACHE_DIR / f"SP1_{season_code}.parquet"
    if parquet_path.exists():
        return pd.read_parquet(parquet_path)

    url = f"https://www.football-data.co.uk/mmz4281/{season_code}/SP1.csv"
    df_fd = pd.read_csv(url)
    df_fd["Date"] = pd.to_datetime(df_fd["Date"], dayfirst=True, errors="coerce")

    # Guarda cache (requiere pyarrow instalado)
    df_fd.to_parquet(parquet_path, index=False)
    return df_fd

def suggest_team_matches(unmatched_names, candidate_names, n=3):
    cand_list = sorted(set(candidate_names))
    out = {}
    for t in sorted(set(unmatched_names)):
        out[t] = get_close_matches(t, cand_list, n=n, cutoff=0.6)
    return out

print("Celda 1 OK ✅")


Celda 1 OK ✅


In [2]:
fd_frames = []
season_map = {}

for s in seasons:
    code = understat_season_to_fd_code(s)
    season_map[s] = code
    df_fd = load_fd_sp1(code)
    df_fd["understat_season"] = s
    df_fd["fd_season_code"] = code
    fd_frames.append(df_fd)

odds_fd = pd.concat(fd_frames, ignore_index=True)

need_cols = ["Date", "HomeTeam", "AwayTeam", "B365H", "B365D", "B365A"]
missing = [c for c in need_cols if c not in odds_fd.columns]
if missing:
    raise ValueError(f"Faltan columnas en Football-Data: {missing}")

odds_1x2 = odds_fd[need_cols].dropna(subset=need_cols).copy()

odds_1x2["match_date"] = pd.to_datetime(odds_1x2["Date"], dayfirst=True, errors="coerce").dt.date
odds_1x2["home_team_clean"] = odds_1x2["HomeTeam"].map(map_team_fd_to_understat)
odds_1x2["away_team_clean"] = odds_1x2["AwayTeam"].map(map_team_fd_to_understat)

print("Seasons (Understat -> FootballData code):", season_map)
print("odds_1x2 shape:", odds_1x2.shape)
display(odds_1x2.head(3))
print("Teams FD (mapped) sample:", sorted(set(odds_1x2["home_team_clean"]))[:15])


Seasons (Understat -> FootballData code): {'2018': '1819', '2019': '1920', '2020': '2021', '2021': '2122', '2022': '2223', '2023': '2324', '2024': '2425', '2025': '2526'}
odds_1x2 shape: (2888, 9)


,Date,HomeTeam,AwayTeam,B365H,B365D,B365A,match_date,home_team_clean,away_team_clean
0,2018-08-17,Betis,Levante,1.66,4.0,5.0,2018-08-17,Real Betis,Levante
1,2018-08-17,Girona,Valladolid,1.75,3.6,5.0,2018-08-17,Girona,Real Valladolid
2,2018-08-18,Barcelona,Alaves,1.11,10.0,21.0,2018-08-18,Barcelona,Alaves


Teams FD (mapped) sample: ['Alaves', 'Almeria', 'Athletic Club', 'Atletico Madrid', 'Barcelona', 'Cadiz', 'Celta Vigo', 'Eibar', 'Elche', 'Espanyol', 'Getafe', 'Girona', 'Granada', 'Huesca', 'Las Palmas']


In [3]:
import soccerdata as sd

understat = sd.Understat("ESP-La Liga", seasons)
df = understat.read_schedule()

print("df cargado ✅")
print("shape:", df.shape)
display(df.head(3))


[02/12/26 21:42:41] INFO     No custom team name replacements found. You can configure these in       ]8;id=761721;file:///Users/mariovelezcanizares/Library/Python/3.9/lib/python/site-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=946178;file:///Users/mariovelezcanizares/Library/Python/3.9/lib/python/site-packages/soccerdata/_config.py#92\92]8;;\
                             /Users/mariovelezcanizares/soccerdata/config/teamname_replacements.json.              

                    INFO     No custom league dict found. You can configure additional leagues in    ]8;id=14109;file:///Users/mariovelezcanizares/Library/Python/3.9/lib/python/site-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=381856;file:///Users/mariovelezcanizares/Library/Python/3.9/lib/python/site-packages/soccerdata/_config.py#198\198]8;;\
                             /Users/mariovelezcanizares/soccerdata/config/league_dict.json.                        

                    INFO     Saving cached data to                                                   ]8;id=517585;file:///Users/mariovelezcanizares/Library/Python/3.9/lib/python/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=24578;file:///Users/mariovelezcanizares/Library/Python/3.9/lib/python/site-packages/soccerdata/_common.py#249\249]8;;\
                             /Users/mariovelezcanizares/soccerdata/data/Understat                                  

[2026-02-12 21:42:41] INFO     TLSLibrary:_load_library:397 - Successfully loaded TLS library: /Users/mariovelezcanizares/Library/Python/3.9/lib/python/site-packages/tls_requests/bin/tls-client-darwin-arm64-1.13.1.dylib


                    INFO     Successfully loaded TLS library:                                      ]8;id=16798;file:///Users/mariovelezcanizares/Library/Python/3.9/lib/python/site-packages/tls_requests/models/libraries.py\libraries.py]8;;\:]8;id=993448;file:///Users/mariovelezcanizares/Library/Python/3.9/lib/python/site-packages/tls_requests/models/libraries.py#397\397]8;;\
                             /Users/mariovelezcanizares/Library/Python/3.9/lib/python/site-package                 
                             s/tls_requests/bin/tls-client-darwin-arm64-1.13.1.dylib                               

                    WARNING  /Users/mariovelezcanizares/Library/Python/3.9/lib/python/site-packages ]8;id=546708;file:///Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/warnings.py\warnings.py]8;;\:]8;id=213982;file:///Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/warnings.py#109\109]8;;\
                             /soccerdata/_common.py:143: UserWarning: Season id "2021" is                          
                             ambiguous: interpreting as "20-21"                                                    
                               warnings.warn(msg, stacklevel=1)                                                    
                                                                                                                   

df cargado ✅
shape: (3040, 17)


league_id  season_id  \
league      season game                                                     
ESP-La Liga 1819   2018-08-17 Girona-Real Valladolid         4       2018   
                   2018-08-17 Real Betis-Levante             4       2018   
                   2018-08-18 Barcelona-Alaves               4       2018   

                                                      game_id  \
league      season game                                         
ESP-La Liga 1819   2018-08-17 Girona-Real Valladolid     9957   
                   2018-08-17 Real Betis-Levante         9958   
                   2018-08-18 Barcelona-Alaves           9961   

                                                                    date  \
league      season game                                                    
ESP-La Liga 1819   2018-08-17 Girona-Real Valladolid 2018-08-17 21:15:00   
                   2018-08-17 Real Betis-Levante     2018-08-17 23:15:00   
                   2018-08-18 Barcelona-Alaves       2018-08-18 23:15:00   

                                                      home_team_id  \
league      season game                                              
ESP-La Liga 1819   2018-08-17 Girona-Real Valladolid           223   
                   2018-08-17 Real Betis-Levante               153   
                   2018-08-18 Barcelona-Alaves                 148   

                                                      away_team_id  \
league      season game                                              
ESP-La Liga 1819   2018-08-17 Girona-Real Valladolid           231   
                   2018-08-17 Real Betis-Levante               151   
                   2018-08-18 Barcelona-Alaves                 158   

                                                       home_team  \
league      season game                                            
ESP-La Liga 1819   2018-08-17 Girona-Real Valladolid      Girona   
                   2018-08-17 Real Betis-Levante      Real Betis   
                   2018-08-18 Barcelona-Alaves         Barcelona   

                                                            away_team  \
league      season game                                                 
ESP-La Liga 1819   2018-08-17 Girona-Real Valladolid  Real Valladolid   
                   2018-08-17 Real Betis-Levante              Levante   
                   2018-08-18 Barcelona-Alaves                 Alaves   

                                                     away_team_code  \
league      season game                                               
ESP-La Liga 1819   2018-08-17 Girona-Real Valladolid            VAL   
                   2018-08-17 Real Betis-Levante                LEV   
                   2018-08-18 Barcelona-Alaves                  ALA   

                                                     home_team_code  \
league      season game                                               
ESP-La Liga 1819   2018-08-17 Girona-Real Valladolid            GIR   
                   2018-08-17 Real Betis-Levante                BET   
                   2018-08-18 Barcelona-Alaves                  BAR   

                                                      home_goals  away_goals  \
league      season game                                                        
ESP-La Liga 1819   2018-08-17 Girona-Real Valladolid           0           0   
                   2018-08-17 Real Betis-Levante               0           3   
                   2018-08-18 Barcelona-Alaves                 3           0   

                                                       home_xg   away_xg  \
league      season game                                                    
ESP-La Liga 1819   2018-08-17 Girona-Real Valladolid  0.626716  0.030934   
                   2018-08-17 Real Betis-Levante       1.00338   2.67481   
                   2018-08-18 Barcelona-Alaves         3.26753  0.248353   

                                                      is_

In [4]:
# CELDA 4 — Preparar df_stats (Understat limpio)
df_stats = df.copy()

# Llave de fecha (sin hora)
df_stats["match_date"] = pd.to_datetime(df_stats["date"], errors="coerce").dt.date

# Llaves de equipos (limpios)
df_stats["home_team_clean"] = df_stats["home_team"].astype(str).str.strip()
df_stats["away_team_clean"] = df_stats["away_team"].astype(str).str.strip()

# Filtrado recomendado si existen
if "has_data" in df_stats.columns:
    df_stats = df_stats[df_stats["has_data"] == True].copy()
if "is_result" in df_stats.columns:
    df_stats = df_stats[df_stats["is_result"] == True].copy()

# Mantener solo filas con datos clave
df_stats = df_stats.dropna(subset=[
    "match_date", "home_team_clean", "away_team_clean",
    "home_goals", "away_goals", "home_xg", "away_xg"
]).copy()

print("df_stats creado ✅", df_stats.shape)
display(df_stats[["match_date","home_team","away_team","home_goals","away_goals","home_xg","away_xg"]].head(5))


df_stats creado ✅ (2888, 20)


match_date  \
league      season game                                              
ESP-La Liga 1819   2018-08-17 Girona-Real Valladolid    2018-08-17   
                   2018-08-17 Real Betis-Levante        2018-08-17   
                   2018-08-18 Barcelona-Alaves          2018-08-18   
                   2018-08-18 Celta Vigo-Espanyol       2018-08-18   
                   2018-08-18 Villarreal-Real Sociedad  2018-08-18   

                                                         home_team  \
league      season game                                              
ESP-La Liga 1819   2018-08-17 Girona-Real Valladolid        Girona   
                   2018-08-17 Real Betis-Levante        Real Betis   
                   2018-08-18 Barcelona-Alaves           Barcelona   
                   2018-08-18 Celta Vigo-Espanyol       Celta Vigo   
                   2018-08-18 Villarreal-Real Sociedad  Villarreal   

                                                              away_team  \
league      season game                                                   
ESP-La Liga 1819   2018-08-17 Girona-Real Valladolid    Real Valladolid   
                   2018-08-17 Real Betis-Levante                Levante   
                   2018-08-18 Barcelona-Alaves                   Alaves   
                   2018-08-18 Celta Vigo-Espanyol              Espanyol   
                   2018-08-18 Villarreal-Real Sociedad    Real Sociedad   

                                                        home_goals  \
league      season game                                              
ESP-La Liga 1819   2018-08-17 Girona-Real Valladolid             0   
                   2018-08-17 Real Betis-Levante                 0   
                   2018-08-18 Barcelona-Alaves                   3   
                   2018-08-18 Celta Vigo-Espanyol                1   
                   2018-08-18 Villarreal-Real Sociedad           1   

                                                        away_goals   home_xg  \
league      season game                                                        
ESP-La Liga 1819   2018-08-17 Girona-Real Valladolid             0  0.626716   
                   2018-08-17 Real Betis-Levante                 3   1.00338   
                   2018-08-18 Barcelona-Alaves                   0   3.26753   
                   2018-08-18 Celta Vigo-Espanyol                1  0.428686   
                   2018-08-18 Villarreal-Real Sociedad           2   1.39716   

                                                         away_xg  
league      season game                                           
ESP-La Liga 1819   2018-08-17 Girona-Real Valladolid    0.030934  
                   2018-08-17 Real Betis-Levante         2.67481  
                   2018-08-18 Barcelona-Alaves          0.248353  
                   2018-08-18 Celta Vigo-Espanyol       0.647286  
                   2018-08-18 Villarreal-Real Sociedad  0.410646

In [5]:
# CELDA 5 — Merge stats + odds
df_merged = df_stats.merge(
    odds_1x2[["match_date", "home_team_clean", "away_team_clean", "B365H", "B365D", "B365A"]],
    on=["match_date", "home_team_clean", "away_team_clean"],
    how="left"
)

has_odds = ~df_merged[["B365H","B365D","B365A"]].isna().any(axis=1)
print(f"Match rate cuotas 1X2: {has_odds.mean():.2%} ({has_odds.sum()} / {len(df_merged)})")

display(df_merged.loc[has_odds, ["match_date","home_team","away_team","B365H","B365D","B365A"]].head(5))

# Reporte de equipos no matcheados
teams_understat = set(df_stats["home_team_clean"]).union(set(df_stats["away_team_clean"]))
teams_fd_mapped = set(odds_1x2["home_team_clean"]).union(set(odds_1x2["away_team_clean"]))

unmatched_understat = sorted(teams_understat - teams_fd_mapped)
print("\nEquipos Understat que NO matchean con Football-Data (después de TEAM_MAP):")
print(unmatched_understat, f"(total={len(unmatched_understat)})")


Match rate cuotas 1X2: 95.26% (2751 / 2888)


,match_date,home_team,away_team,B365H,B365D,B365A
0,2018-08-17,Girona,Real Valladolid,1.75,3.6,5.0
1,2018-08-17,Real Betis,Levante,1.66,4.0,5.0
2,2018-08-18,Barcelona,Alaves,1.11,10.0,21.0
3,2018-08-18,Celta Vigo,Espanyol,1.85,3.5,4.5
4,2018-08-18,Villarreal,Real Sociedad,2.04,3.4,3.8



Equipos Understat que NO matchean con Football-Data (después de TEAM_MAP):
['Real Oviedo', 'SD Huesca'] (total=2)


In [6]:
# CELDA 5 — Merge stats + odds
df_merged = df_stats.merge(
    odds_1x2[["match_date", "home_team_clean", "away_team_clean", "B365H", "B365D", "B365A"]],
    on=["match_date", "home_team_clean", "away_team_clean"],
    how="left"
)

has_odds = ~df_merged[["B365H","B365D","B365A"]].isna().any(axis=1)
print(f"Match rate cuotas 1X2: {has_odds.mean():.2%} ({has_odds.sum()} / {len(df_merged)})")

display(df_merged.loc[has_odds, ["match_date","home_team","away_team","B365H","B365D","B365A"]].head(5))

# Reporte de equipos no matcheados
teams_understat = set(df_stats["home_team_clean"]).union(set(df_stats["away_team_clean"]))
teams_fd_mapped = set(odds_1x2["home_team_clean"]).union(set(odds_1x2["away_team_clean"]))

unmatched_understat = sorted(teams_understat - teams_fd_mapped)
print("\nEquipos Understat que NO matchean con Football-Data (después de TEAM_MAP):")
print(unmatched_understat, f"(total={len(unmatched_understat)})")


Match rate cuotas 1X2: 95.26% (2751 / 2888)


,match_date,home_team,away_team,B365H,B365D,B365A
0,2018-08-17,Girona,Real Valladolid,1.75,3.6,5.0
1,2018-08-17,Real Betis,Levante,1.66,4.0,5.0
2,2018-08-18,Barcelona,Alaves,1.11,10.0,21.0
3,2018-08-18,Celta Vigo,Espanyol,1.85,3.5,4.5
4,2018-08-18,Villarreal,Real Sociedad,2.04,3.4,3.8



Equipos Understat que NO matchean con Football-Data (después de TEAM_MAP):
['Real Oviedo', 'SD Huesca'] (total=2)


In [7]:
# CELDA 6 — Dataset final para ML
df_merged["result"] = np.where(
    df_merged["home_goals"] > df_merged["away_goals"], "H",
    np.where(df_merged["home_goals"] < df_merged["away_goals"], "A", "D")
)

df_merged["pH_raw"] = 1 / df_merged["B365H"]
df_merged["pD_raw"] = 1 / df_merged["B365D"]
df_merged["pA_raw"] = 1 / df_merged["B365A"]

s = df_merged["pH_raw"] + df_merged["pD_raw"] + df_merged["pA_raw"]
df_merged["pH"] = df_merged["pH_raw"] / s
df_merged["pD"] = df_merged["pD_raw"] / s
df_merged["pA"] = df_merged["pA_raw"] / s
df_merged["vig"] = s - 1

df_merged["xg_diff"] = df_merged["home_xg"] - df_merged["away_xg"]
df_merged["total_goals"] = df_merged["home_goals"] + df_merged["away_goals"]

has_odds = ~df_merged[["B365H","B365D","B365A"]].isna().any(axis=1)
df_model = df_merged.loc[has_odds].copy()

print("df_model ✅", df_model.shape)
display(df_model.head(5))

teams_model = sorted(set(df_model["home_team_clean"]).union(set(df_model["away_team_clean"])))
print("\nN° equipos:", len(teams_model))
print("Equipos:", teams_model)

print("\nDistribución H/D/A:")
print(df_model["result"].value_counts(normalize=True))


df_model ✅ (2751, 33)


,league_id,season_id,game_id,date,home_team_id,away_team_id,home_team,away_team,away_team_code,home_team_code,...,result,pH_raw,pD_raw,pA_raw,pH,pD,pA,vig,xg_diff,total_goals
0,4,2018,9957,2018-08-17 21:15:00,223,231,Girona,Real Valladolid,VAL,GIR,...,D,0.571429,0.277778,0.200000,0.544629,0.264750,0.190620,0.049206,0.595782,0
1,4,2018,9958,2018-08-17 23:15:00,153,151,Real Betis,Levante,LEV,BET,...,A,0.602410,0.250000,0.200000,0.572410,0.237550,0.190040,0.052410,-1.67143,3
2,4,2018,9961,2018-08-18 23:15:00,148,158,Barcelona,Alaves,ALA,BAR,...,H,0.900901,0.100000,0.047619,0.859212,0.095373,0.045415,0.048520,3.019177,3
3,4,2018,9959,2018-08-18 19:15:00,152,141,Celta Vigo,Espanyol,ESP,CEL,...,D,0.540541,0.285714,0.222222,0.515548,0.272504,0.211948,0.048477,-0.2186,2
4,4,2018,9960,2018-08-18 21:15:00,154,140,Villarreal,Real Sociedad,SOC,VIL,...,A,0.490196,0.294118,0.263158,0.467980,0.280788,0.251232,0.047472,0.986514,3



N° equipos: 26
Equipos: ['Alaves', 'Almeria', 'Athletic Club', 'Atletico Madrid', 'Barcelona', 'Cadiz', 'Celta Vigo', 'Eibar', 'Elche', 'Espanyol', 'Getafe', 'Girona', 'Granada', 'Las Palmas', 'Leganes', 'Levante', 'Mallorca', 'Osasuna', 'Rayo Vallecano', 'Real Betis', 'Real Madrid', 'Real Sociedad', 'Real Valladolid', 'Sevilla', 'Valencia', 'Villarreal']

Distribución H/D/A:
result
H    0.445656
A    0.286441
D    0.267903
Name: proportion, dtype: float64


In [8]:
# CELDA 7 — Asegurar df_model listo para análisis/modelos
if "df_model" not in globals():
    raise NameError("No existe df_model. Ejecuta las celdas anteriores (merge + features) antes de seguir.")

use_cols = ["match_date","home_team_clean","away_team_clean","home_goals","away_goals",
            "home_xg","away_xg","xg_diff","total_goals",
            "B365H","B365D","B365A","pH","pD","pA","vig","result"]

missing = [c for c in use_cols if c not in df_model.columns]
if missing:
    raise KeyError(f"Faltan columnas en df_model: {missing}")

dfA = df_model.dropna(subset=["B365H","B365D","B365A","pH","pD","pA","xg_diff","result"]).copy()
dfA = dfA.sort_values("match_date").reset_index(drop=True)

print("dfA (análisis/modelos) shape:", dfA.shape)
display(dfA.head(3))


dfA (análisis/modelos) shape: (2751, 33)


,league_id,season_id,game_id,date,home_team_id,away_team_id,home_team,away_team,away_team_code,home_team_code,...,result,pH_raw,pD_raw,pA_raw,pH,pD,pA,vig,xg_diff,total_goals
0,4,2018,9957,2018-08-17 21:15:00,223,231,Girona,Real Valladolid,VAL,GIR,...,D,0.571429,0.277778,0.200000,0.544629,0.264750,0.190620,0.049206,0.595782,0
1,4,2018,9958,2018-08-17 23:15:00,153,151,Real Betis,Levante,LEV,BET,...,A,0.602410,0.250000,0.200000,0.572410,0.237550,0.190040,0.052410,-1.67143,3
2,4,2018,9961,2018-08-18 23:15:00,148,158,Barcelona,Alaves,ALA,BAR,...,H,0.900901,0.100000,0.047619,0.859212,0.095373,0.045415,0.048520,3.019177,3


In [9]:
# CELDA 8 — Correlación odds vs xG
corr = dfA[["pH","pD","pA","B365H","B365D","B365A","xg_diff","home_xg","away_xg"]].corr(numeric_only=True)

print("Correlación con xg_diff (positivo = favorito local en xG):")
display(corr[["xg_diff"]].sort_values("xg_diff", ascending=False))


Correlación con xg_diff (positivo = favorito local en xG):


,xg_diff
xg_diff,1.000000
home_xg,0.792268
pH,0.524703
B365A,0.450764
B365D,0.241415
pD,-0.227817
B365H,-0.440062
pA,-0.514891
away_xg,-0.712444
